In [2]:
# libraries
from dotenv import load_dotenv
import os
from tavily import TavilyClient

# load environment variables from .env file
_ = load_dotenv("../.env")

In [3]:
# connect
client = TavilyClient(api_key=os.environ.get("TAVILY_API_KEY"))

In [4]:
# run search
result = client.search("What is in Nvidia's new Blackwell GPU?", include_answer=True)

# print the answer
result["answer"]

'The new Nvidia Blackwell GPU architecture features groundbreaking compute performance and optimized for large-scale AI models. It includes the second-generation Transformer Engine, confidential computing to protect sensitive data, and fifth-generation NVLink with 50GB/sec per link bandwidth, supporting up to 576 GPUs for seamless scaling of large AI workloads. The Blackwell B200 is built on a TSMC 4NP node and is designed to meet the demands of generative AI.'

### Regular search

In [5]:
# choose location (try to change to your own city!)

city = "bangkok"

query = f"""
    what is the current weather in {city}?
    Should I travel there today?
    "weather.com"
"""

In [6]:
import requests
from bs4 import BeautifulSoup
from duckduckgo_search import DDGS
import re

ddg = DDGS()


def search(query, max_results=6):
    try:
        results = ddg.text(query, max_results=max_results)
        return [i["href"] for i in results]
    except Exception as e:
        print(f"returning previous results due to exception reaching ddg.")
        results = [  # cover case where DDG rate limits due to high deeplearning.ai volume
            "https://weather.com/weather/today/l/USCA0987:1:US",
            "https://weather.com/weather/hourbyhour/l/54f9d8baac32496f6b5497b4bf7a277c3e2e6cc5625de69680e6169e7e38e9a8",
        ]
        return results


for i in search(query):
    print(i)

https://weather.com/en-LC/weather/tenday/l/61d235a12c8f0b158c472bb5cf4a6a2d17b42270c214e7285c48666e57f21864?traffic_source=footerNav_Tenday
https://weather.com/weather/hourbyhour/l/c2ed3e9b1efee2ca7085681e732f8f432a46b8bf935f5abf08ae066a11f2be32
https://weather.com/weather/today/l/Bangor+ME?canonicalCityId=1bed76693e82f04bba756ceae52fcb63ecf5eb99fde38578ea9ecae34a0de94e
https://weather.com/weather/today/l/Loganville+GA?canonicalCityId=4f25fbe452b99ea6512a083b951ee10d2d7cdb9ef7e6e0d21355f862c6902b16
https://weather.com/weather/tenday/l/Rialto+CA?canonicalCityId=daee3b97cb98f252cc3b31d9cb1582ca4ab4145d4049138d8a35e76069444cb7
https://weather.com/weather/today/l/Brentwood+CA?canonicalCityId=e6f3776c3d84993470cf3760f3164154dbd9e39064a6c6a4e80753762f470e63


In [7]:
def scrape_weather_info(url):
    """Scrape content from the given URL"""
    if not url:
        return "Weather information could not be found."

    # fetch data
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        return "Failed to retrieve the webpage."

    # parse result
    soup = BeautifulSoup(response.text, "html.parser")
    return soup

In [10]:
# use DuckDuckGo to find websites and take the first result
url = search(query)[1]

# scrape first wesbsite
soup = scrape_weather_info(url)

print(f"Website: {url}\n\n")
print(str(soup.body)[:50000])  # limit long outputs

Website: http://www.google.com/search?hl=en&q=what+is+the+current+weather+in+bangkok?+Should+I+travel+there+today?+"weather.com"


<body><noscript><style>table,div,span,p{display:none}</style><meta content="0;url=/httpservice/retry/enablejs?sei=EseyZ5bJGaTv4-EPq6yUuAo" http-equiv="refresh"/><div style="display:block">Please click <a href="/httpservice/retry/enablejs?sei=EseyZ5bJGaTv4-EPq6yUuAo">here</a> if you are not redirected within a few seconds.</div></noscript><script nonce="9EG1WpPyWx7yMxxpDG8L2A">//# sourceMappingURL=data:application/json;charset=utf-8;base64,eyJ2ZXJzaW9uIjogMywic291cmNlcyI6WyIiXSwic291cmNlc0NvbnRlbnQiOlsiICJdLCJuYW1lcyI6WyJjbG9zdXJlRHluYW1pY0J1dHRvbiJdLCJtYXBwaW5ncyI6IkFBQUE7QUFBQTtBQUFBO0FBQUE7QUFBQTtBQUFBO0FBQUEifQ==
(function(){function Z(P){return P}var g=function(P,J,X,m,W,C,d,S,T,E,L,h){for(L=(h=68,67);;)try{if(h==95)break;else if(h==J)L=83,T=S.createPolicy(C,{createHTML:r,createScript:r,createScriptURL:r}),h=52;else{if(h==52)return L=67,T;if(h==68)T=d,S

In [11]:
# extract text
weather_data = []
for tag in soup.find_all(["h1", "h2", "h3", "p"]):
    text = tag.get_text(" ", strip=True)
    weather_data.append(text)

# combine all elements into a single string
weather_data = "\n".join(weather_data)

# remove all spaces from the combined text
weather_data = re.sub(r"\s+", " ", weather_data)

print(f"Website: {url}\n\n")
print(weather_data)

Website: http://www.google.com/search?hl=en&q=what+is+the+current+weather+in+bangkok?+Should+I+travel+there+today?+"weather.com"





### Agentic Search

In [12]:
# run search
result = client.search(query, max_results=1)

# print first result
data = result["results"][0]["content"]

print(data)

{'location': {'name': 'Bangkok', 'region': 'Krung Thep', 'country': 'Thailand', 'lat': 13.75, 'lon': 100.5167, 'tz_id': 'Asia/Bangkok', 'localtime_epoch': 1739769664, 'localtime': '2025-02-17 12:21'}, 'current': {'last_updated_epoch': 1739769300, 'last_updated': '2025-02-17 12:15', 'temp_c': 33.3, 'temp_f': 91.9, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 9.6, 'wind_kph': 15.5, 'wind_degree': 176, 'wind_dir': 'S', 'pressure_mb': 1013.0, 'pressure_in': 29.91, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 49, 'cloud': 25, 'feelslike_c': 35.9, 'feelslike_f': 96.7, 'windchill_c': 33.4, 'windchill_f': 92.0, 'heatindex_c': 36.1, 'heatindex_f': 96.9, 'dewpoint_c': 19.5, 'dewpoint_f': 67.1, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 9.9, 'gust_mph': 11.1, 'gust_kph': 17.8}}


In [13]:
import json
from pygments import highlight, lexers, formatters

# parse JSON
parsed_json = json.loads(data.replace("'", '"'))

# pretty print JSON with syntax highlighting
formatted_json = json.dumps(parsed_json, indent=4)
colorful_json = highlight(
    formatted_json, lexers.JsonLexer(), formatters.TerminalFormatter()
)

print(colorful_json)

{
    "location": {
        "name": "Bangkok",
        "region": "Krung Thep",
        "country": "Thailand",
        "lat": 13.75,
        "lon": 100.5167,
        "tz_id": "Asia/Bangkok",
        "localtime_epoch": 1739769664,
        "localtime": "2025-02-17 12:21"
    },
    "current": {
        "last_updated_epoch": 1739769300,
        "last_updated": "2025-02-17 12:15",
        "temp_c": 33.3,
        "temp_f": 91.9,
        "is_day": 1,
        "condition": {
            "text": "Partly cloudy",
            "icon": "//cdn.weatherapi.com/weather/64x64/day/116.png",
            "code": 1003
        },
        "wind_mph": 9.6,
        "wind_kph": 15.5,
        "wind_degree": 176,
        "wind_dir": "S",
        "pressure_mb": 1013.0,
        "pressure_in": 29.91,
        "precip_mm": 0.0,
        "precip_in": 0.0,
        "humidity": 49,
        "cloud": 25,
        "feelslike_c": 35.9,
        "feelslike_f": 96.7,
        "windchill_c": 33.4,
        "windchill_f": 92.0,
        